# The Best Location For A Sports Bar In Toronto

## Introduction/Business Problem

According to research done here, https://www.brightlocal.com/research/local-business-travel-times/, people are willing to drive less distance to go to a specific pub/bar than any other business category besides the gym or yoga / alternative therapies. The location of a sports bar is one of the biggest determinants if it will get enough traffic to pay all of its expenses and eventually turn a profit. Sports bars have become much more popular over the last handful of years and finding a location that still has plenty of demand can be tricky.

Our objective is to recommend the best locations in Toronto to open a new sports bar.

## Data

Coordinate data of Toronto postal codes is in http://cocl.us/Geospatial_data. The data base for Toronto postal code population and total private dwellings are from https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&S=22&O=A.

According to the research from Bright Local, on average, people are willing to drive about 14 minutes to a pub of their choice. In a city, we will assume you can drive about 8 km (or 5 miles) in 14 minutes. Using the Foursquare API we will determine how many sports bars are within a 5 mile radius of each postal code. We will then use these numbers to determine the number of sports bars per 5 mile radius there are for each postal code.

We will generate a table with rows corresponding to neighborhood, and columns with
- postal code
- longitude
- lattitude
- population
- sports bars in 5 mile radius
- sports bars per capita

The neighborhood with the least amount of sports bars per capita should be the ones with the most potential demand for a new sports bar.